#Добавление библиотек

In [1]:
import sqlite3
import re

#Создание БД

In [7]:
conn = sqlite3.connect('task1.sqlite') 
cursor = conn.cursor()

cursor.execute('''CREATE TABLE actors (
                id INTEGER PRIMARY KEY,
                name TEXT )
''')

cursor.execute('''CREATE TABLE movies (
                id INTEGER PRIMARY KEY,
                title TEXT )
''')

cursor.execute('''CREATE TABLE actor_movie ( 
               actor_id INTEGER, 
               movie_id INTEGER, 
               FOREIGN KEY (actor_id) REFERENCES actors(id), 
               FOREIGN KEY (movie_id) REFERENCES movies(id) )
''')

conn.commit()

conn.close()

OperationalError: table actors already exists

#Достаем информацию

In [2]:
connect_netflix = sqlite3.connect('netflix.sqlite')
cursor_netflix = connect_netflix.cursor()

cursor_netflix.execute('select title, "cast" from netflix_titles')

# Получаем результаты запроса и сохраняем их в переменную
data = cursor_netflix.fetchall()

connect_netflix.close()

#Преобразуем в удобный вид

In [3]:
data_new = []
unique_actors = set()
for row in data:
    movie, cast_str = row
    cast = re.split(r' ,|, |,', cast_str)
    unique_actors = unique_actors.union(set(cast))
    data_new.append((movie, cast))

#Заполнение фильмами

In [8]:
connect_movies = sqlite3.connect('task1.sqlite')
cursor_movies = connect_movies.cursor()

for row in data_new:
    movie, actors = row
    cursor_movies.execute("INSERT INTO movies VALUES (NULL, ?)", (movie,))

connect_movies.commit()
connect_movies.close()


#Заполение актерами

In [9]:
connect_actors = sqlite3.connect('task1.sqlite')
cursor_actors = connect_actors.cursor()

for actor in unique_actors:
    cursor_actors.execute("INSERT INTO actors VALUES (NULL, ?)", (actor,))

connect_actors.commit()
connect_actors.close()

#Соединение многие-ко-многим

In [4]:
connect_task1 = sqlite3.connect('task1.sqlite')

cursor_movies = connect_task1.cursor()
cursor_actors = connect_task1.cursor()
cursor_actor_movie = connect_task1.cursor()

for i in data_new:
    movie, actors = i
    cursor_movies.execute('SELECT id FROM movies WHERE title = ?', (movie,))
    movie_id = cursor_movies.fetchone()
    for actor in actors:
        cursor_actors.execute('SELECT id FROM actors WHERE name = ?', (actor,))
        actor_id = cursor_actors.fetchone()   
        cursor_actor_movie.execute("INSERT INTO actor_movie (actor_id, movie_id) VALUES (?, ?)", (actor_id[0], movie_id[0]))  # добавление записи в таблицу


connect_task1.commit()
connect_task1.close()

#поиск актеров

In [5]:
conn = sqlite3.connect('task1.sqlite')
cursor = conn.cursor()

cursor.execute('''
    SELECT a1.name AS actor1_name, a2.name AS actor2_name, COUNT(*) AS movies_together
    FROM actor_movie am1
    JOIN actor_movie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
    JOIN actors a1 ON am1.actor_id = a1.id
    JOIN actors a2 ON am2.actor_id = a2.id
    GROUP BY actor1_name, actor2_name
    ORDER BY movies_together DESC
    LIMIT 1
''')

result = cursor.fetchone()

print("Самая часто работающая друг с другом пара актеров:")
print(f"{result[0]} и {result[1]} - сыграли вместе в {result[2]} фильмах")

conn.close()


Самая часто работающая друг с другом пара актеров:
Robb Wells и John Paul Tremblay - сыграли вместе в 15 фильмах
